### Extract directory structure from commits and summarize rate of change

In [ ]:
from config import repo_url, export_directory
from common import collect_commits

In [ ]:
commits = collect_commits(repo_url)

In [ ]:
import os.path

class Directory:
    
    ## class Directory maintains a static snapshot of a directory structure
    def __init__(self, name=''):
        self.sub_directories = {}
        self.files = {}
        self.size = 0
        self.freeze = False
        self.name = name
        
    def add_file(self, path, file, size = 1):
        
        dirname = os.path.dirname(path)
        
        if dirname == '':
            self.files[path] = file
        else:
            sub_directory = path.split('/')[0]
            if sub_directory not in self.sub_directories: #create new directory
                self.sub_directories[sub_directory] = Directory(name=sub_directory)
            self.sub_directories[sub_directory].add_file('/'.join(path.split('/')[1:]), file, size = size)
        
        self.size += size
    
    def remove_file(self, path, size = 1):
        
        dirname = os.path.dirname(path)
        
        if dirname == '':
            del self.files[path]
        else:
            sub_directory = path.split('/')[0]
            self.sub_directories[sub_directory].remove_file('/'.join(path.split('/')[1:]), size = size)
            
        self.size -= size
    
    def rename_file(self, old_path, new_path, file, size = 1):
        
        self.remove_file(old_path, size = size)
        self.add_file(new_path, file, size = size)
        
    def modify_file(self, path, size_change = 0):
    
        self.size += size_change
        
        dirname = os.path.dirname(path)
        if dirname != '':
            sub_directory = path.split('/')[0]
            self.sub_directories[sub_directory].modify_file('/'.join(path.split('/')[1:]), size_change = size_change)
        
    def clean_print(self, level=0):
        
        ## print a depth first tranversal of the tree
        print(' ' * level + self.name)
        for directoryname in self.sub_directories:
            self.sub_directories[directoryname].clean_print(level = level + 1)
        for filename in self.files:
            print(' ' * (level + 1) + filename)
        
    def to_json_tree(self):
        
        pass

